## Packages install

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
%matplotlib inline
import pandas as pd
import re
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp

## Import data files

### 1.1 Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1.2 Load data file

In [ ]:
path = '/content/drive/My Drive/Intro to Data Science 2023 Fall/Project 1/'
data = path + "movieReplicationSet.csv"
df=pd.read_csv(data)

In [ ]:
df

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,3.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,0,0
1093,3.0,4.0,NaN,NaN,4.0,4.0,2.5,NaN,3.5,3.5,...,5.0,3.0,5.0,5.0,5.0,6.0,5.0,1.0,0,0
1094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,6.0,3.0,1.0,6.0,6.0,4.0,2.0,1.0,0,0
1095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,4.0,3.0,3.0,1.0,1.0,0,1


## Start doing data analysis and hypothesis tests

### 1) Are movies that are more popular (operationalized as having more ratings) rated higher than movies that are less popular?

In [ ]:
# Select the first 400 columns
first_400_columns = df.iloc[:, :400]

# Calculate the popularity and average of the 400 movies
popularity = first_400_columns.count()
avg_rating = first_400_columns.mean()

# Create two groups with high popularity and low popularity
popularity_avg = {'Popularity': popularity,
        'Average Ratings': avg_rating}

df1 = pd.DataFrame(popularity_avg)

# Step 1: Calculate the median of the 'Popularity' column
popularity_median = df1['Popularity'].median()

# Step 2: Compare each movie's popularity to the median and assign to groups
unpopular_movies = df1[df1['Popularity'] <= popularity_median]
popular_movies = df1[df1['Popularity'] > popularity_median]

In [ ]:
# Step 3: Calculate inter-group variance of ratings
unpopular_var=unpopular_movies['Average Ratings'].var()
popular_var=popular_movies['Average Ratings'].var()
print('unpopular_var',unpopular_var)
print('popular_var',popular_var)

# Step 4: Perform the t-test
t_stat, p_value = stats.ttest_ind(unpopular_movies['Average Ratings'], popular_movies['Average Ratings'],alternative='less')
print('t_stat, p_value',t_stat, p_value)

# Interpret the results
alpha = 0.005  # Set your significance level
if p_value < alpha:
    print("There is a significant difference between high-popularity and low-popularity movie ratings.")
else:
    print("There is no significant difference between high-popularity and low-popularity movie ratings.")

unpopular_var 0.05357798836736424
popular_var 0.0850024321309087
t_stat, p_value -17.7560492698737 1.1348265138282423e-52
There is a significant difference between high-popularity and low-popularity movie ratings.


### 2) Are movies that are newer rated differently than movies that are older? [Hint: Do a median split of year of release to contrast movies in terms of whether they are old or new]


In [ ]:
# Question 2
def get_movie_year_pair(movies):
    movie_year_pair = []
    for movie in df.columns[:400]:
        res = re.match(".*?\((\d{4})\)", movie)
        movie_year_pair.append((movie,int(res.group(1))))
    return list(map(lambda x: x[0],sorted(movie_year_pair, key=lambda x: x[1])))

        
def split_time_wise(df):
    # 1. Extract the timestamp
    movies = df.columns[:400]
    movie_sorted_by_year = get_movie_year_pair(movies)
    length = len(movie_sorted_by_year)
    
    # 2. Split the dataset by release date
    statistics = dict(df[df.columns[:400]].mean(axis = 0))
    older_group_statistics = [statistics[movie] for movie in movie_sorted_by_year[:length // 2]]
    newer_group_statistics = [statistics[movie] for movie in movie_sorted_by_year[length // 2:]]
    
    variance_group_old = np.array(older_group_statistics).var()
    variance_group_new = np.array(newer_group_statistics).var()
    print(variance_group_old, variance_group_new)
    return [older_group_statistics,newer_group_statistics]


def perform_t_test(df):
    [older_group, newer_group] = split_time_wise(df)
    t1,p1 = stats.ttest_ind(older_group, newer_group)
    print(t1,p1)
    
perform_t_test(df)

### 3) Is enjoyment of ‘Shrek (2001)’ gendered, i.e. do male and female viewers rate it differently?


In [ ]:
# Split the ratings of "Shrek" by gender
shrek_ratings_male = df[df['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 2]['Shrek (2001)']
shrek_ratings_female = df[df['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 1]['Shrek (2001)']

# Calculate intra variance
male_var=shrek_ratings_male.var()
female_var=shrek_ratings_female.var()
print('male_var',male_var)
print('female_var',female_var)

t_stat, p_value = stats.ttest_ind(shrek_ratings_male, shrek_ratings_female, nan_policy='omit',equal_var=False)
print('t_stat, p_value',t_stat, p_value)

# Interpret the results
alpha = 0.005  # Set your significance level
if p_value < alpha:
    print("There is a significant difference between high-popularity and low-popularity movie ratings.")
else:
    print("There is no significant difference between high-popularity and low-popularity movie ratings.")

male_var 0.6805843706777317
female_var 0.8218267169230881
t_stat, p_value -1.1558907155973421 0.24834907946281018
There is no significant difference between high-popularity and low-popularity movie ratings.


### 4) What proportion of movies are rated differently by male and female viewers?

In [ ]:
# Question 4
# Helper function for question 4,6,8
def t_test_for_movie(movie, attribute, split):
    group_1 = movie[movie[attribute] == split[0]].dropna(axis=0)
    group_2 = movie[movie[attribute] == split[1]].dropna(axis=0)
    
    
    group_1_stats = list(group_1.iloc[:,0])
    group_2_stats = list(group_2.iloc[:,0])
    
    group_1_var = group_1.iloc[:,0].var()
    group_2_var = group_2.iloc[:,0].var()
    
    print(group_1_var, group_2_var)

    t, p = stats.ttest_ind(group_1_stats, group_2_stats)
    return t, p
    
def calculate_proportion(df, attribute, split):
    
    count = 0
    for movie in df.columns[:400]:
        t, p = t_test_for_movie(df[[movie, attribute]], attribute, split)
        if p < 0.005:
            count += 1
    
    return count / 400
    

calculate_proportion(df,"Gender identity (1 = female; 2 = male; 3 = self-described)", [1,2])

### 5) Do people who are only children enjoy ‘The Lion King (1994)’ more than people with siblings?

In [ ]:
# Split the ratings of "Shrek" by gender
lion_ratings_siblings = df[df['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 0]['The Lion King (1994)']
lion_ratings_nosiblings = df[df['Are you an only child? (1: Yes; 0: No; -1: Did not respond)'] == 1]['The Lion King (1994)']

# Calculate intra variance
siblings_var=lion_ratings_siblings.var()
nosiblings_var=lion_ratings_nosiblings.var()
print('siblings_var',siblings_var)
print('nosiblings_var',nosiblings_var)

t_stat, p_value = stats.ttest_ind(lion_ratings_siblings, lion_ratings_nosiblings, nan_policy='omit',alternative='less')
print('t_stat, p_value',t_stat, p_value)

# Interpret the results
alpha = 0.005  # Set your significance level
if p_value < alpha:
    print("There is a significant difference between high-popularity and low-popularity movie ratings.")
else:
    print("There is no significant difference between high-popularity and low-popularity movie ratings.")

siblings_var 0.5158031260392417
nosiblings_var 0.6666445916114792
t_stat, p_value 2.053888996058986 0.9798664723686588
There is no significant difference between high-popularity and low-popularity movie ratings.


In [ ]:
df.iloc[:, 475]

0       0
1       0
2       1
3       0
4       1
       ..
1092    0
1093    0
1094    0
1095    0
1096    0
Name: Are you an only child? (1: Yes; 0: No; -1: Did not respond), Length: 1097, dtype: int64

### 6) What proportion of movies exhibit an “only child effect”, i.e. are rated different by viewers with siblingsvs. those without?

In [ ]:
# Question 6
calculate_proportion(df,"Are you an only child? (1: Yes; 0: No; -1: Did not respond)", [1,0])

### 7):  Do people who like to watch movies socially enjoy ‘The Wolf of Wall Street (2013)’ more than those who prefer to watch them alone?


In [ ]:
# Split the ratings of "Shrek" by gender
wolf_ratings_social = df[df['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 0]['The Wolf of Wall Street (2013)']
wolf_ratings_alone = df[df['Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)'] == 1]['The Wolf of Wall Street (2013)']

# Calculate intra variance
wolf_ratings_social_var=wolf_ratings_social.var()
wolf_ratings_alone_var=wolf_ratings_alone.var()
print('wolf_ratings_social_var',wolf_ratings_social_var)
print('wolf_ratings_alone_var',wolf_ratings_alone_var)

t_stat, p_value = stats.ttest_ind(wolf_ratings_social, wolf_ratings_alone, nan_policy='omit',alternative='greater')
print('t_stat, p_value',t_stat, p_value)

# Interpret the results
alpha = 0.005  # Set your significance level
if p_value < alpha:
    print("There is a significant difference between high-popularity and low-popularity movie ratings.")
else:
    print("There is no significant difference between high-popularity and low-popularity movie ratings.")

wolf_ratings_social_var 0.8483271375464686
wolf_ratings_alone_var 0.7567021083242457
t_stat, p_value -1.567873874504994 0.9413054316716771
There is no significant difference between high-popularity and low-popularity movie ratings.


### 8) What proportion of movies exhibit such a “social watching” effect?

In [ ]:
# Question 8
calculate_proportion(df,"Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)", [0,1])

### 9) Is the ratings distribution of ‘Home Alone (1990)’ different than that of ‘Finding Nemo (2003)’? 

In [ ]:
# Question 9
def perform_ks_test(data_1,data_2):
    
    ks, p = ks_2samp(data_1,data_2)
    return ks, p
    
    
def compare_distribution(df):
    group_1 = df['Home Alone (1990)'].dropna()
    group_2 = df['Finding Nemo (2003)'].dropna()
    
    ks, p = perform_ks_test(group_1, group_2)
        
    print(ks, p)

compare_distribution(df)  

### 10) There are ratings on movies from several franchises ([‘Star Wars’, ‘Harry Potter’, ‘The Matrix’, ‘Indiana Jones’, ‘Jurassic Park’, ‘Pirates of the Caribbean’, ‘Toy Story’, ‘Batman’]) in this dataset. How many of these are of inconsistent quality, as experienced by viewers? [Hint: You can use the keywords in quotation marks featured in this question to identify the movies that are part of each franchise]

In [ ]:
# Question 10
def franchise_anova(df, franchise):
    movies = df.columns[:400][df.columns[:400].str.contains(franchise)]
    num_of_movies = len(movies)
    dropped_movies_row_wise = df[movies].dropna()
    groups = [dropped_movies_row_wise.iloc[:, index] for index in range(num_of_movies)]

    f, p = stats.f_oneway(*groups)
    
    return f,p
    
def compute_inconsistency(df, franchise_list):
    count = 0
    for franchise in franchise_list:
        f, p = franchise_anova(df, franchise)
        print(franchise, f,p)
        if p < 0.005:
            count += 1
    print(count)
    return count / len(franchise_list)
    
franchise_list = ['Star Wars','Harry Potter','The Matrix','Indiana Jones','Jurassic Park','Pirates of the Caribbean','Toy Story','Batman']
compute_inconsistency(df, franchise_list)

### Extra credit:

In [ ]:
df['Average Rating'] = df.iloc[:, :400].mean(axis=1)
grouped = df.groupby('Gender identity (1 = female; 2 = male; 3 = self-described)')
ratings_female = df[df['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 1]['Average Rating']
ratings_male = df[df['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 2]['Average Rating']
ratings_other = df[df['Gender identity (1 = female; 2 = male; 3 = self-described)'] == 3]['Average Rating']

In [ ]:
female_var=ratings_female.var()
male_var=ratings_male.var()
other_var=ratings_other.var()
print('female_var,male_var,other_var',female_var,male_var,other_var)

female_var,male_var,other_var 0.2204158439117208 0.2666415455767475 0.16607817233199895


In [ ]:
f_statistic, p_value = stats.f_oneway(ratings_male, ratings_female, ratings_other)
print('f_statistic, p_value',f_statistic, p_value)

f_statistic, p_value 1.677625761642752 0.18730800328635697
